In [ ]:
import os
import numpy as np
import pydicom
import png
source_folder = r'/home/aimlab/Documents/Courses/Deep_learning/Project/DICOM_PNG_converter/dcm'
output_folder = r'/home/aimlab/Documents/Courses/Deep_learning/Project/DICOM_PNG_converter/png'


def dicom2png(source_folder, output_folder):
    list_of_files = os.listdir(source_folder)
    f = open('labels.csv','w')
    # for file in list_of_files:
    #     f.write(file)
    #     f.write('\n')
    # f.close()
    for file in list_of_files:
        f.write(file)
        f.write('\n')
        try:
            ds = pydicom.read_file(os.path.join(source_folder,file))
            # Convert to float to avoid overflow or underflow losses.
            # image_2d = ds.pixel_array.astype(float)
            shape = ds.pixel_array.shape

            # Load dimensions based on the number of rows, columns
            x_dim, y_dim = (int(ds.Rows), int(ds.Columns))
            # load pixel value
            xray = ds.pixel_array
            # flip the pixels upside-down so that image is not inverted
            # xray = np.flipud(xray)
            if ds.PhotometricInterpretation == "MONOCHROME1":
                xray = np.amax(xray) - xray
            xray = xray - np.min(xray)
            xray = xray / np.max(xray)
            xray = (xray * 255).astype(np.uint8)
            # Write the PNG file
            print(file)

            file = os.path.splitext(file)[0]
            with open(os.path.join(output_folder,file)+'.png' , 'wb') as png_file:
                w = png.Writer(shape[1], shape[0], greyscale=True)
                w.write(png_file, xray)
        except:
            print('Could not convert: ', file)
    f.close()
dicom2png(source_folder, output_folder)

In [166]:
source_folder = r'/home/aimlab/Documents/Courses/Deep_learning/Project/DICOM_PNG_converter/dcm'
output_folder = r'/home/aimlab/Documents/Courses/Deep_learning/Project/DICOM_PNG_converter/png'
def dicom2png(source_folder, output_folder):
    list_of_files = os.listdir(source_folder)
    f = open('labels.csv','w')
    f.write('Filename')
    f.write('\n')
    for file in list_of_files:
        f.write(file)
        f.write('\n')
    f.close()
dicom2png(source_folder, output_folder)

In [167]:
## This part is for using the labels and exctracting the columns values of the original csv file and creating a new csv file.
import pandas as pd 
df = pd.read_csv(r'./labels.csv')
org_df = pd.read_csv(r'./metadata_global_v2.csv', sep= ';')
# print(org_df)

In [177]:
# print(list(org_df.columns))
org_df.columns

Index(['Filename', 'StudyDate', 'Modality', 'Columns', 'Rows', 'Manufacturer',
       'PhotometricInterpretation', 'BrixiaScore', 'BrixiaScoreGlobal',
       'ConsensusTestset', 'Subject', 'StudyId', 'Sex',
       'AgeAtStudyDateFiveYear'],
      dtype='object')

In [176]:
df.columns

Index(['Filename'], dtype='object')

In [179]:
merged_inner = pd.merge(left=df,right=org_df, left_on='Filename', right_on='Filename')
merged_inner.shape
merged_inner

,Filename,StudyDate,Modality,Columns,Rows,Manufacturer,PhotometricInterpretation,BrixiaScore,BrixiaScoreGlobal,ConsensusTestset,Subject,StudyId,Sex,AgeAtStudyDateFiveYear
0,98728340646032925.dcm,20200312,CR,2866,2350,SIEMENS,MONOCHROME2,11011,4,0,P0830,A0703,M,14
1,166414673190348825.dcm,20200325,DX,3032,2520,CARESTREAM HEALTH,MONOCHROME1,1021,4,0,P0093,A0447,M,13
2,746021306010419641.dcm,20200318,DX,3032,2520,CARESTREAM HEALTH,MONOCHROME1,122122,10,0,P1301,A1952,M,14
3,759417224465559435.dcm,20200322,CR,2866,1748,SIEMENS,MONOCHROME2,132123,12,0,P0182,A4255,F,16
4,849569894482494578.dcm,20200330,CR,2866,2350,SIEMENS,MONOCHROME2,232131,12,0,P1523,A2815,M,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,351036371249274070.dcm,20200319,DX,2539,1776,Carestream Health,MONOCHROME2,1012,4,0,P0799,A0431,M,4
302,221170870669898830.dcm,20200329,CR,2698,2110,SIEMENS,MONOCHROME2,11013,6,0,P2024,A3611,F,17
303,79376362217868614.dcm,20200330,CR,2866,2350,SIEMENS,MONOCHROME2,31032,9,0,P1552,A1338,M,14
304,120990829650033034.dcm,20200314,CR,2866,2350,SIEMENS,MONOCHROME2,3,3,0,P1659,A0235,M,14


In [185]:
merged_inner.to_csv('Image_labels.csv', sep = ';', encoding='utf-8', index=False)